In [6]:
columns =  ["day_year",
            "day_week", 
            "week", 
            "borough", 
            "fhv", 
            
            "2019_night", 
            "2019_morn",
            "2019_arvo",
            "2019_even",
            "2020_night", 
            "2020_morn",
            "2020_arvo",
            "2020_even",
            
            'borough_case_count', 
            'borough_case_count_7day_avg', 
            'borough_hospitalized_count', 
            'borough_hospitalized_count_7day_avg',
            'borough_death_count', 
            'borough_all_death_count_7day_avg',
            
            "stadium_capcity", 
            "restaurants", 
            "high_schools", 
            "indoor_religious" , 
            "phase_1"
    ]

In [7]:
# ██████╗  ██████╗  ███╗   ██╗ ███████╗ ██╗  ██████╗  ██╗   ██╗ ██████╗   █████╗  ████████╗ ██╗  ██████╗  ███╗   ██╗ 
#██╔════╝ ██╔═══██╗ ████╗  ██║ ██╔════╝ ██║ ██╔════╝  ██║   ██║ ██╔══██╗ ██╔══██╗ ╚══██╔══╝ ██║ ██╔═══██╗ ████╗  ██║ 
#██║      ██║   ██║ ██╔██╗ ██║ █████╗   ██║ ██║  ███╗ ██║   ██║ ██████╔╝ ███████║    ██║    ██║ ██║   ██║ ██╔██╗ ██║ 
#██║      ██║   ██║ ██║╚██╗██║ ██╔══╝   ██║ ██║   ██║ ██║   ██║ ██╔══██╗ ██╔══██║    ██║    ██║ ██║   ██║ ██║╚██╗██║ 
#╚██████╗ ╚██████╔╝ ██║ ╚████║ ██║      ██║ ╚██████╔╝ ╚██████╔╝ ██║  ██║ ██║  ██║    ██║    ██║ ╚██████╔╝ ██║ ╚████║ 
# ╚═════╝  ╚═════╝  ╚═╝  ╚═══╝ ╚═╝      ╚═╝  ╚═════╝   ╚═════╝  ╚═╝  ╚═╝ ╚═╝  ╚═╝    ╚═╝    ╚═╝  ╚═════╝  ╚═╝  ╚═══╝ 
# global variables that are chagned often

#█▀▀ █▀▀ █▄ █ █▀▀ █▀█ ▄▀█ █   
#█▄█ ██▄ █ ▀█ ██▄ █▀▄ █▀█ █▄▄ 
#
#variable for skipping all the plotting code for debugging processing
plotting = True

#chose while taxi types to be processed
process_ywl = True
process_grn = True
process_fhv = True
process_hvf = False   #setting this to true adds 24hr processing

#limits data size and processes to be quicker (for testing only) 
testing = False
start_month = 3        #testing var default 2
number_of_months = 5   #testing var default 11
start_day = 60         #testing var default 32ish
number_of_days = 100     #testing var default 366


#█▀▄ ▄▀█ ▀█▀ ▄▀█    █▀▄ █ █▀█ 
#█▄▀ █▀█  █  █▀█    █▄▀ █ █▀▄ 
#

#main data directory
data_dir = "/mnt/e/2021/Applied Data Science/Project 1/Data/"

#intermediate file names
processed_data_file = "_processed_data_"
tally_s1_file = "taxi_data_tallyed_s1.csv"
tally_s2_file = "taxi_data_tallyed_s2_neg.csv"

In [8]:
#█▀█ █   █▀█ ▀█▀ ▀█▀ █ █▄ █ █▀▀    █   █ █▄▄ █▀ 
#█▀▀ █▄▄ █▄█  █   █  █ █ ▀█ █▄█    █▄▄ █ █▄█ ▄█ 
# #setting up libs

import numpy as np
from scipy.stats import entropy
from math import e

from collections import OrderedDict

import random as random
np.random.seed(1)
from pprint import pprint

import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
plt.show() 
plt.rcParams['figure.figsize'] = (15.0, 15.0)
plt.show() 
plt.rcParams.update({'font.size': 22})

plt.rcParams['axes.grid'] = False

from scipy.stats import mode, chi2_contingency, pearsonr 
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, BaggingClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn import datasets
from sklearn import svm

import warnings

warnings.filterwarnings("ignore")

In [9]:
from statsmodels.formula.api import *
from glmnet import ElasticNet

In [10]:
# ██████╗  ██╗       ██████╗  ██████╗   █████╗  ██╗         ██╗   ██╗  █████╗  ██████╗  ███████╗ 
#██╔════╝  ██║      ██╔═══██╗ ██╔══██╗ ██╔══██╗ ██║         ██║   ██║ ██╔══██╗ ██╔══██╗ ██╔════╝ 
#██║  ███╗ ██║      ██║   ██║ ██████╔╝ ███████║ ██║         ██║   ██║ ███████║ ██████╔╝ ███████╗ 
#██║   ██║ ██║      ██║   ██║ ██╔══██╗ ██╔══██║ ██║         ╚██╗ ██╔╝ ██╔══██║ ██╔══██╗ ╚════██║ 
#╚██████╔╝ ███████╗ ╚██████╔╝ ██████╔╝ ██║  ██║ ███████╗     ╚████╔╝  ██║  ██║ ██║  ██║ ███████║ 
# ╚═════╝  ╚══════╝  ╚═════╝  ╚═════╝  ╚═╝  ╚═╝ ╚══════╝      ╚═══╝   ╚═╝  ╚═╝ ╚═╝  ╚═╝ ╚══════╝ 
# v2.2.1
# global variabels which can be tweeked

#█    ▀  █▀▀▄ █▀▀█ █▀▀█ █▀▀█ █  █ █▀▀ 
#█   ▀█▀ █▀▀▄ █▄▄▀ █▄▄█ █▄▄▀ █▄▄█ ▀▀█ 
#▀▀▀ ▀▀▀ ▀▀▀  ▀ ▀▀ ▀  ▀ ▀ ▀▀ ▄▄▄█ ▀▀▀ 
# imporint libarys used throughout

import time
import datetime

import pandas as pd
from numpy import log, sqrt
import seaborn as sns
import matplotlib.pyplot as plt
from math import e

import os.path
from os.path import getsize
from urllib.request import urlretrieve

from pyspark.sql.functions import rand 
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.sql.functions import col
from pyspark.sql import SparkSession

import warnings

#variable for skipping all the plotting code for debugging processing
start_time = time.perf_counter()
start_time_og = time.perf_counter()


#▀█▀ ▄▀█ ▀▄▀ █    █▀▄ ▄▀█ ▀█▀ ▄▀█ 
# █  █▀█ █ █ █    █▄▀ █▀█  █  █▀█ 
#

#file location
file_base = "_tripdata_"

#indexs for each of the main data set types
ywl_i = 0 
grn_i = 1
fhv_i = 2
hvf_i = 3

#used to identify which taxis to process
process_taxi = [process_ywl, process_grn, process_fhv, process_hvf]

#names related to each cab type in order
taxi_file_names = ["yellow","green","fhv","fhvhv"]
full_names = ["yellow cabs", "green cabs", "for higher vheciles", "for higher high volume"]

#column names for each cab type in order
column_names = [
    ['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag', 'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount', 'congestion_surcharge'],
    ['VendorID', 'lpep_pickup_datetime', 'lpep_dropoff_datetime', 'store_and_fwd_flag', 'RatecodeID', 'PULocationID', 'DOLocationID', 'passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'ehail_fee', 'improvement_surcharge', 'total_amount', 'payment_type', 'trip_type', 'congestion_surcharge'],
    ['dispatching_base_num', 'pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID', 'SR_Flag'],
    ['hvfhs_license_num', 'dispatching_base_num', 'pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID', 'SR_Flag']
]

#2018 fhv files which was giving me trouble so added this schema for it
bad_fhv_schema = [2, 2018, ['Pickup_DateTime','DropOff_datetime', 'PULocationID', 'DOLocationID', 'SR_Flag','dispatching_base_number']]


#aliases for pickup time and dropoff time
pickup_time = ['tpep_pickup_datetime','lpep_pickup_datetime','pickup_datetime','pickup_datetime','Pickup_DateTime']
dropoff_time = ['tpep_dropoff_datetime','lpep_dropoff_datetime','dropoff_datetime','dropoff_datetime','DropOff_datetime']

#irrelavent collumns that can be dropped
irrelavent_columns = ['dispatching_base_number','DOtime','hvfhs_license_num','dispatching_base_num','VendorID','store_and_fwd_flag','extra','MTA_tax','improvement_surcharge','tip_amount','tolls_amount', 'ehail_fee', 'improvement_surcharge', 'congestion_surcharge']

#array, for all the taxi count colls
all_taxi_cols = [
        "2019_night", 
        "2019_morn",
        "2019_arvo",
        "2019_even",
        "2020_night", 
        "2020_morn",
        "2020_arvo",
        "2020_even"
        ]

#array for 2020 taxi count colls
taxi_cols_2020 = [
        "2020_night", 
        "2020_morn",
        "2020_arvo",
        "2020_even"]


#▀█▀ ▄▀█ ▀▄▀ █    █   █▀█ █▀▀ ▄▀█ ▀█▀ █ █▀█ █▄ █ █▀ 
# █  █▀█ █ █ █    █▄▄ █▄█ █▄▄ █▀█  █  █ █▄█ █ ▀█ ▄█ 
#

#location processing variables
#locations that will be removed as considered invalid
bad_locations = [1,132,138,264,265]

#location of the location id file
location_file = "taxi+_zone_lookup.csv"

#list of boroughs names as in the location file
boroughs_locations = ["Bronx","Brooklyn","Manhattan","Queens","Staten Island","EWR","Unknown"]



#█▀▄ █▀█ █ █ █ █▄ █ █   █▀█ ▄▀█ █▀▄ 
#█▄▀ █▄█ ▀▄▀▄▀ █ ▀█ █▄▄ █▄█ █▀█ █▄▀ 
#

#download directory 
download_dir = data_dir

#taxi data base url
taxi_url = f"https://s3.amazonaws.com/nyc-tlc/trip+data/"

#covid cases
covid_url = "https://data.cityofnewyork.us/api/views/rc75-m7u3/rows.csv?accessType=DOWNLOAD"
    
#location data
location_url = "https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv"

#█▀▄ ▄▀█ ▀█▀ █▀▀    █▀▄ █▀▀ █▀▀ █▀ 
#█▄▀ █▀█  █  ██▄    █▄▀ ██▄ █▀  ▄█ 
#
months_in_year = 12

#years of data used
years = [2019, 2020]

#feb 2019 was when fhvhv file started
fhvhv_started = [2,2019] 

#late feb 2020 was when the first case of covid was observed in NYC
covid_started = [2,2020] 

#preprocessing arrays

#hours each day to seperate each count
day_time_breakdown = [0,6,12,18]

#day of the year covid started on
covid_start_day = int(datetime.datetime.strptime(
    " ".join([str(int) for int in covid_started]), 
    "%m %Y").__format__("%j"))

#pretty self explanitory
days_in_2020 = 366



#█▀█ █ █ ▀█▀ █   █ █▀▀ █▀█ █▀ 
#█▄█ █▄█  █  █▄▄ █ ██▄ █▀▄ ▄█ 
# values for calculating outliers

#all vals must be withing the .99 percent of data
outlier_percent = (1-.99)/2

#buffer for graphing and removing outliers from graphs
fare_amount_graph_buffer = 15

#accuracy when calculating the percentile
percentile_accuracy = 0.00001



#█▀▀ █▀█ █ █ █ █▀▄    █▀▄ █▀▀ █▀▀ █▀ 
#█▄▄ █▄█ ▀▄▀ █ █▄▀    █▄▀ ██▄ █▄▄ ▄█ 
#

#files
covid_restrictions_file = "covid_restrictions.csv"
covid_cases_file = "COVID-19_Daily_Counts_of_Cases__Hospitalizations__and_Deaths.csv"

#list of boroughs as named in the covid file (same order)
covid_boroughs =["bx","bk","mn","qn","si"]



#█▀ █▀▀ ▀█▀ █ █ █▀█    █▀▀ █▀█ █▄ █ █▀▄ █ ▀█▀ █ █▀█ █▄ █ █▀ 
#▄█ ██▄  █  █▄█ █▀▀    █▄▄ █▄█ █ ▀█ █▄▀ █  █  █ █▄█ █ ▀█ ▄█ 
#

#plotting is using a yellow data set so must skip plotting if no yellow
if plotting:
    if not (process_taxi[ywl_i] and process_taxi[grn_i]):
        plotting = False
        
        
#if in a testing mode, sets up variables so file processes correctly         
if testing:
    
    #changes the download dir to not overwrite data
    download_dir =  data_dir.split("1/Data/")[0] + "1/TestDownload/"
    
    #changes some timing variabels to anaylise only what is needed for testing
    covid_started[0] = start_month
    months_in_year = start_month + number_of_months - 1
    graph_month = start_month
    days_in_2020 = start_day + number_of_days
else:
    
    #if not testing graphing month is 5 (so may)
    graph_month = 5
    
    
    
#█▀▄▀█ █ █▀ █▀▀    █▀▀ █ █ █▄ █ █▀▀ 
#█ ▀ █ █ ▄█ █▄▄    █▀  █▄█ █ ▀█ █▄▄ 
# specialized functions

#sets seed for when its used
seed = 1

# log function to deal with integer count data, added conditions to set any negative
# or 0 values to be 0
def logf(x):
    return (log(x) if x > 0 else 0)

#applys a log log others 0
def loglogf(x):
    return (log(log(x)) if x > e else 0)

#applys log log to an array
def loglogfa(arr):
    return [loglogf(x) for x in arr]

In [11]:

#main data directory
data_dir = "/mnt/e/2021/Applied Data Science/Project 1/Data/"

#list of boroughs names as in the location file
boroughs = ["Bronx","Brooklyn","Manhattan","Queens","Staten Island"]

df = pd.read_csv(data_dir + tally_s2_file)
df

,Unnamed: 0,day_year,day_week,week,borough,fhv,2019_night,2019_morn,2019_arvo,2019_even,...,borough_case_count_7day_avg,borough_all_case_count_7day_avg,borough_hospitalized_count_7day_avg,borough_death_count_7day_avg,borough_all_death_count_7day_avg,stadium_capcity,restaurants,high_schools,indoor_religious,phase_1
0,0,32,6,4,0,0,1.676556,1.860428,1.884553,1.810529,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0,1.0,1.0,1.0
1,1,32,6,4,0,1,2.293099,2.309731,2.338324,2.366162,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0,1.0,1.0,1.0
2,2,32,6,4,1,0,2.112605,2.052718,2.136567,2.166455,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0,1.0,1.0,1.0
3,3,32,6,4,1,1,2.394665,2.390935,2.436074,2.467458,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0,1.0,1.0,1.0
4,4,32,6,4,2,0,2.374448,2.400022,2.454497,2.454562,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3315,3315,363,1,52,2,1,2.340907,2.421726,2.470082,2.459379,...,1.806491,1.847767,1.276345,0.475885,0.475885,0.0,0.0,0.0,0.5,1.0
3316,3316,363,1,52,3,0,1.835632,1.893093,1.969277,1.995207,...,1.912448,1.948918,1.449898,0.834032,0.834032,0.0,0.0,0.0,0.5,1.0
3317,3317,363,1,52,3,1,2.297639,2.370983,2.391751,2.381531,...,1.912448,1.948918,1.449898,0.834032,0.834032,0.0,0.0,0.0,0.5,1.0
3318,3318,363,1,52,4,0,0.000000,0.583198,0.000000,0.094048,...,1.739366,1.774470,1.061385,0.475885,0.475885,0.0,0.0,0.0,0.5,1.0


In [92]:
taxi_19_new = []
taxi_20_new = []

for col in all_taxi_cols:
    strip = col.split("_")
    new_name = strip[1] + strip[0][2:4]
    if(strip[0][2:4] == "20"):
        taxi_20_new.append(new_name)
    else:
        taxi_19_new.append(new_name)
    df[new_name] = df[col]
    
df.columns

Index(['Unnamed: 0', 'day_year', 'day_week', 'week', 'borough', 'fhv',
       '2019_night', '2019_morn', '2019_arvo', '2019_even', '2020_night',
       '2020_morn', '2020_arvo', '2020_even', 'borough_case_count',
       'borough_probable_case_count', 'borough_hospitalized_count',
       'borough_death_count', 'borough_probable_death_count',
       'borough_case_count_7day_avg', 'borough_all_case_count_7day_avg',
       'borough_hospitalized_count_7day_avg', 'borough_death_count_7day_avg',
       'borough_all_death_count_7day_avg', 'stadium_capcity', 'restaurants',
       'high_schools', 'indoor_religious', 'phase_1', 'night19', 'morn19',
       'arvo19', 'even19', 'night20', 'morn20', 'arvo20', 'even20'],
      dtype='object')

In [93]:
yCOLS = taxi_20_new[0]
s = set(all_taxi_cols)
xCOLS = [x for x in columns if x not in s]
xCOLS.extend(taxi_19_new)

# standardize (by calculating the zscore) so our data has mean 0 and var 1
# alternatively, you can use sklearn's StandardScalar

from scipy.stats import zscore
#df_standard = df_filtered[xCOLS].astype(float).apply(zscore)
df_filtered = df
df_standard = df[xCOLS]
df_x = df[xCOLS]

xCOLS

['day_year',
 'day_week',
 'week',
 'borough',
 'fhv',
 'borough_case_count',
 'borough_case_count_7day_avg',
 'borough_hospitalized_count',
 'borough_hospitalized_count_7day_avg',
 'borough_death_count',
 'borough_all_death_count_7day_avg',
 'stadium_capcity',
 'restaurants',
 'high_schools',
 'indoor_religious',
 'phase_1',
 'night19',
 'morn19',
 'arvo19',
 'even19']

In [105]:


lasso_fit = ElasticNet()
fit = lasso_fit.fit(df_standard.values, df_filtered[yCOLS].values)

AttributeError: 'ElasticNet' object has no attribute 'summary'

In [95]:
xCOLS

['day_year',
 'day_week',
 'week',
 'borough',
 'fhv',
 'borough_case_count',
 'borough_case_count_7day_avg',
 'borough_hospitalized_count',
 'borough_hospitalized_count_7day_avg',
 'borough_death_count',
 'borough_all_death_count_7day_avg',
 'stadium_capcity',
 'restaurants',
 'high_schools',
 'indoor_religious',
 'phase_1',
 'night19',
 'morn19',
 'arvo19',
 'even19']

In [96]:
print(f'Best lambda value for LASSO: {lasso_fit.lambda_best_[0]}')

Best lambda value for LASSO: 0.025397499065444088


In [97]:
pd.DataFrame(index = ['Intercept'] + xCOLS, 
             data= [lasso_fit.intercept_] + list(lasso_fit.coef_), 
             columns = ['Coefficient'])

,Coefficient
Intercept,-0.001912
day_year,0.000000
day_week,0.000000
week,0.000000
borough,0.000000
fhv,0.477183
borough_case_count,-0.025097
borough_case_count_7day_avg,0.000000
borough_hospitalized_count,-0.006648
borough_hospitalized_count_7day_avg,-0.053990


In [98]:
formula = taxi_20_new[0] + " ~ " + " +".join(xCOLS)
formula

'night20 ~ day_year +day_week +week +borough +fhv +borough_case_count +borough_case_count_7day_avg +borough_hospitalized_count +borough_hospitalized_count_7day_avg +borough_death_count +borough_all_death_count_7day_avg +stadium_capcity +restaurants +high_schools +indoor_religious +phase_1 +night19 +morn19 +arvo19 +even19'

In [99]:
#formula = "2020_night"+ " ~ " + " +".join(xCOLS)

model = ols(formula = formula, data= df)

In [104]:
print(model.fit().summary())

                            OLS Regression Results                            
Dep. Variable:                night20   R-squared:                       0.950
Model:                            OLS   Adj. R-squared:                  0.950
Method:                 Least Squares   F-statistic:                     3316.
Date:                Thu, 12 Aug 2021   Prob (F-statistic):               0.00
Time:                        21:55:31   Log-Likelihood:                 1327.7
No. Observations:                3320   AIC:                            -2615.
Df Residuals:                    3300   BIC:                            -2493.
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
In